## Coded By Frank Zhao

In [2]:
import json
from pprint import pprint
import glob
from datetime import datetime
import requests
import pandas as pd
import re

# load json file


In [140]:
## path
json_path =  r'.\data\house_hist_json'
folder = ["\p1_450", "\p451_900", "\p901_1350", "\p1351_1800", "\p1801_2250", "\p2251_2700"]
csv_path = r".\data\hist_price_csv"

In [142]:
all_files = glob.glob(path + folder[0] +"/*.txt")

# extract info


In [4]:
def process_response(response_json):
    '''
    This function is to convert each request result to a dataframe.
    
    1. create an empty list
    2. loop for each response and get details from key 'properties' 
    3. convert details to df
    4. append single df to list
    5. concat the list to one df
    
    '''
    
    # empty list
    convert_list=[]
#     if response_json['properties']: return pd.DataFrame([])
    # loop through each 'properties'
    
    if 'properties' not in response_json.keys():
        return pd.DataFrame(columns = ['event_name',
                                       'date',
                                       'price',
                                       'price_range_min',
                                       'price_range_max',
                                       'price_changed',
                                       'sqft',
                                       'datasource_name',
                                       'source'
                                      ])
    
    if 'property_history' not in response_json['properties'][0].keys():
        return pd.DataFrame(columns = ['event_name',
                                       'date',
                                       'price',
                                       'price_range_min',
                                       'price_range_max',
                                       'price_changed',
                                       'sqft',
                                       'datasource_name',
                                       'source'
                                      ])
        
    
    for col in response_json['properties'][0]['property_history']:
        
        # convert details to dataframe
        single_df = pd.DataFrame.from_dict(col, orient='index').T
        
        # append to list
        convert_list.append(single_df)
        
    # concat to a whole df, null for missing vals
    return pd.concat(convert_list, axis = 0, ignore_index=True, sort=False)

# Combine all together

In [6]:
## path
json_path =  r'.\data\house_hist_json'
folder = ["\p1_450", "\p451_900", "\p901_1350", "\p1351_1800",
          "\p1801_2250", "\p2251_2700", "\p2701_3150", '\p3151_3600',
          '\p3601_4050', '\p4051_4500', '\p4501_4800', '\p4801_9562'
         ]
csv_path = r".\data\hist_price_csv"

In [7]:
%%time
## 1. loop into folders
count = 1

for f in folder:
    print(f"Now in folder: {f}.")
    all_files = glob.glob(json_path + f +"/*.txt") 
    
    # 2. loop in each file
    for file in all_files: # string of directory
        
        # 3. get property id
        p_id = re.findall('(?<=_)[A-Z]\d+(?=.)', file)[0]
        
        # 4.load json file
        with open(file) as ff:
            js = json.load(ff)
            
        # 5. extract historical price
        hist_df = process_response(js)
        
        # 6. insert property id col to df
        hist_df.insert(loc=0, column='property_id', value=p_id)
        
        # 7. export to csv
        hist_df.to_csv(f'{csv_path}{f}\\hist_price_{count}_{p_id}.csv',encoding='utf-8', index=False)
        
        if count % 100 == 0:
            print(f"The {count}th run")
            
        count += 1
        
    

Now in folder: \p1_450.
The 100th run
The 200th run
The 300th run
The 400th run
Now in folder: \p451_900.
The 500th run
The 600th run
The 700th run
The 800th run
The 900th run
Now in folder: \p901_1350.
The 1000th run
The 1100th run
The 1200th run
The 1300th run
Now in folder: \p1351_1800.
The 1400th run
The 1500th run
The 1600th run
The 1700th run
The 1800th run
Now in folder: \p1801_2250.
The 1900th run
The 2000th run
The 2100th run
The 2200th run
Now in folder: \p2251_2700.
The 2300th run
The 2400th run
The 2500th run
The 2600th run
The 2700th run
Now in folder: \p2701_3150.
The 2800th run
The 2900th run
The 3000th run
The 3100th run
Now in folder: \p3151_3600.
The 3200th run
The 3300th run
The 3400th run
The 3500th run
The 3600th run
Now in folder: \p3601_4050.
The 3700th run
The 3800th run
The 3900th run
The 4000th run
Now in folder: \p4051_4500.
The 4100th run
The 4200th run
The 4300th run
The 4400th run
The 4500th run
Now in folder: \p4501_4800.
The 4600th run
The 4700th run
The

# combine all csv to 1 df

In [8]:
%%time
## 1. loop into folders
count = 1

file_list = []
for f in folder:
    print(f"Now in folder: {f}.")
    price_df = glob.glob(csv_path + f +"/*.csv") 
    
    
    # 2. loop in each file
    for filename in price_df: # string of directory
        
        df = pd.read_csv(filename, index_col=None, header=0)
        file_list.append(df)
 
        if count % 500 == 0:
            print(f"The {count}th run")
            
        count += 1

combined_df = pd.concat(file_list, axis = 0, ignore_index=True)

Now in folder: \p1_450.
The 500th run
Now in folder: \p451_900.
The 1000th run
The 1500th run
Now in folder: \p901_1350.
The 2000th run
Now in folder: \p1351_1800.
The 2500th run
Now in folder: \p1801_2250.
Now in folder: \p2251_2700.
The 3000th run
Now in folder: \p2701_3150.
The 3500th run
Now in folder: \p3151_3600.
The 4000th run
Now in folder: \p3601_4050.
The 4500th run
Now in folder: \p4051_4500.
The 5000th run
Now in folder: \p4501_4800.
The 5500th run
Now in folder: \p4801_9562.
The 6000th run
The 6500th run
The 7000th run
The 7500th run
The 8000th run
The 8500th run
The 9000th run
The 9500th run
The 10000th run
Wall time: 1min 2s


In [11]:
# combined_df.to_csv(f'.\data\hist_price_total\hist_price_all.csv',encoding='utf-8', index=False)

In [193]:
## part 2

In [196]:
%%time
## import all csv to one df
count = 1

path = r'.\data\hist_price_csv\p2'
all_files = glob.glob(path + "/*.csv")

## empty list
file_list = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    file_list.append(df)
    
    if count % 200 ==0:
        print(f"The {count}th run")
    count += 1
    
hist_p2 = pd.concat(file_list, axis = 0, ignore_index=True)

hist_p2

The 200th run
The 400th run
The 600th run
The 800th run
The 1000th run
The 1200th run
The 1400th run
The 1600th run
The 1800th run
The 2000th run
The 2200th run
The 2400th run
The 2600th run
The 2800th run
The 3000th run
The 3200th run
The 3400th run
The 3600th run
The 3800th run
The 4000th run
The 4200th run
The 4400th run
The 4600th run
Wall time: 22.6 s


,property_id,event_name,date,price,price_range_min,price_range_max,price_changed,sqft,datasource_name,source,listing,iso_date,previous_event_price,listing_id
0,M3603156744,Listed,2020-07-28T17:00:00Z,439000,NaN,NaN,0,1211,BrightMLS,MLS #PAPH919538,NaN,NaN,NaN,NaN
1,M3603156744,Listed,2019-03-06T17:00:00Z,415000,NaN,NaN,0,1211,BrightMLS,MLS #PAPH723786,NaN,NaN,NaN,NaN
2,M3603156744,Price Changed,2010-07-23T17:00:00Z,331415,NaN,NaN,38990,1211,BrightMLS,MLS #5659515,NaN,NaN,NaN,NaN
3,M3603156744,Listed,2010-02-15T17:00:00Z,292425,NaN,NaN,0,1211,BrightMLS,MLS #5659515,NaN,NaN,NaN,NaN
4,M3603156744,Listed,2008-08-08T17:00:00Z,389900,NaN,NaN,0,1211,BrightMLS,MLS #5400762,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28972,M3052313655,Listed,2020-07-12T17:00:00Z,289900,NaN,NaN,0,2122,BrightMLS,MLS #PAPH913864,NaN,NaN,NaN,NaN
28973,M3052313655,Price Changed,2020-06-29T17:00:00Z,314900,NaN,NaN,-20000,2100,BrightMLS,MLS #PAPH899964,NaN,NaN,NaN,NaN
28974,M3052313655,Price Changed,2020-06-17T17:00:00Z,334900,NaN,NaN,-15000,2100,BrightMLS,MLS #PAPH899964,NaN,NaN,NaN,NaN
28975,M3052313655,Listed,2020-06-01T17:00:00Z,349900,NaN,NaN,0,2100,BrightMLS,MLS #PAPH899964,NaN,NaN,NaN,NaN


In [197]:
# hist_p2.to_csv(f'.\data\hist_price_total\hist_price_part2.csv',encoding='utf-8', index=False)